In [1]:
from preprocess import load_data

In [2]:
x_train, y_train = load_data([1], True)
x_test, y_test = load_data([1])

In [3]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(88, 274944) (128, 274944)
(88, 127808) (128, 127808)


In [4]:
import numpy as np

x_train = np.reshape(x_train.T, (x_train.shape[1] // 64, 64, 88))
y_train = np.reshape(y_train.T, (y_train.shape[1] // 64, 64, 128))
x_test = np.reshape(x_test.T, (x_test.shape[1] // 64, 64, 88))
y_test = np.reshape(y_test.T, (y_test.shape[1] // 64, 64, 128))

In [5]:
from lstm import get_text_model

args = get_text_model()
args.model.fit(
    x_train, y_train,
    epochs=args.epochs, 
    batch_size=args.batch_size,
    validation_data=(x_test, y_test)
)

2022-12-01 11:13:50.019266: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
43/43 [==============================] - 6s 104ms/step - loss: 0.3061 - binary_accuracy: 0.9217 - val_loss: 0.0922 - val_binary_accuracy: 0.9808
Epoch 2/5
43/43 [==============================] - 4s 94ms/step - loss: 0.1047 - binary_accuracy: 0.9745 - val_loss: 0.0852 - val_binary_accuracy: 0.9808
Epoch 3/5
43/43 [==============================] - 4s 88ms/step - loss: 0.1011 - binary_accuracy: 0.9745 - val_loss: 0.0832 - val_binary_accuracy: 0.9808
Epoch 4/5
43/43 [==============================] - 4s 86ms/step - loss: 0.0995 - binary_accuracy: 0.9745 - val_loss: 0.0821 - val_binary_accuracy: 0.9808
Epoch 5/5
43/43 [==============================] - 4s 95ms/step - loss: 0.0982 - binary_accuracy: 0.9745 - val_loss: 0.0813 - val_binary_accuracy: 0.9808
